In [1]:
import json
import requests
from pandas.io.json import json_normalize
import datetime
from datetime import datetime, timedelta, time, date
import pandas as pd
import calendar
import numpy as np

In [2]:
from credentials import screendoor_key, screendoor_project_id

In [30]:
last_year = datetime.now().year - 1
this_month = datetime.now().month
last_12_months = datetime(last_year, this_month, 1)
last_12_months = last_12_months.strftime('%Y-%m-%d')

In [25]:
def call_screendoor_api():
    
    records = '10'
    
    #read in the open data customer support requests using the screendoor api
    api_request = requests.get('https://screendoor.dobt.co/api/projects/'+screendoor_project_id+'/responses?per_page='+records+'&v=0&api_key='+screendoor_key)
    screendoor_raw_data = api_request.json()
    
    #create a dataframe
    screendoor_df = pd.DataFrame(screendoor_raw_data)
    
    #extract the nested dictionary 'responses'
    normalize_responses = json_normalize(screendoor_df['responses'])
    
    #join both dataframs 
    concat_df = pd.concat([screendoor_df,normalize_responses], axis=1,join_axes=[screendoor_df.index])
    
    #reformat data types
    concat_df.submitted_at = pd.to_datetime(concat_df.submitted_at,format='%Y-%m-%dT%H:%M:%S.%fZ')
    concat_df.updated_at = pd.to_datetime(concat_df.updated_at,format='%Y-%m-%dT%H:%M:%S.%fZ')
    
    #key fields in the underlying dictionary do not make sense. rename some of them
    concat_df['request_type'] = concat_df['fygvab39']
    
    #create new dataframe with only columns needed 
    screendoor_df = concat_df[['id','submitted_at','updated_at','request_type','status']].copy()

    #create new 'request_type_grouped' column
    status_conditions = [
        (screendoor_df['request_type']=='Request a dataset'),
        (screendoor_df['request_type']=='General inquiry'),
        (screendoor_df['request_type']=='Ask a question about a dataset'),
        (screendoor_df['request_type']=='Report an error in the data')
    ]
    status_choices = ['Request a Dataset', 'General Inquiry', 'Data Question', 'Report Error']
    screendoor_df['request_type_grouped'] = np.select(status_conditions, status_choices, default='Other')
    
    #calculate amount of time between submission and last updated  
    screendoor_df['update_time'] = (screendoor_df.updated_at-screendoor_df.submitted_at) / np.timedelta64(1,'D')


    return screendoor_df

In [26]:
sd = call_screendoor_api()
sd.shape

(10, 7)

In [27]:
sd

,id,submitted_at,updated_at,request_type,status,request_type_grouped,update_time
0,603827,2017-03-02 22:42:31.395,2018-10-15 12:18:35.211,Request a dataset,Closed,Request a Dataset,591.566711
1,606421,2017-03-03 16:01:45.523,2017-03-14 22:32:05.123,Provide an additional resource for this website,Closed,Other,11.271060
2,607093,2017-03-03 19:09:57.162,2017-07-24 20:54:37.076,Ask a question about a dataset,Closed,Data Question,143.072684
3,607458,2017-03-03 20:47:39.628,2017-03-03 22:10:16.727,General inquiry,Closed,General Inquiry,0.057374
4,607485,2017-03-03 21:14:16.189,2018-08-30 14:56:11.139,Request a dataset,Closed,Request a Dataset,544.737442
5,608450,2017-03-04 02:20:30.545,2017-03-08 16:38:24.724,General inquiry,Closed,General Inquiry,4.595766
6,609105,2017-03-04 09:23:48.450,2017-03-14 21:48:19.356,General inquiry,Closed,General Inquiry,10.517024
7,609780,2017-03-04 18:44:53.740,2017-07-24 20:54:53.621,Report an error in the data,Closed,Report Error,142.090276
8,609898,2017-03-04 20:13:14.941,2017-03-08 22:16:45.499,Report an error in the data,Closed,Report Error,4.085770
9,610058,2017-03-04 22:25:18.730,2017-03-08 22:15:55.750,Report an error in the data,Closed,Report Error,3.993484


In [28]:
def group_requests_by_type_and_status(screendoor_df):
    # count requests by type and status (top groups)
    requests_by_status_grouped = screendoor_df[screendoor_df['submitted_at'] >= last_12_months]
    requests_by_status_grouped = requests_by_status_grouped.groupby(['request_type_grouped', 'status'])[['id']].count().reset_index()
    return requests_by_status_grouped

In [32]:
sd_grp = group_requests_by_type_and_status(sd)
sd_grp.shape

(0, 3)